# LightGBM Regressor 

## Mục tiêu
- Xây dựng mô hình LightGBM Regressor để dự đoán tuổi thọ trung bình
- Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`
- Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation

## Giới thiệu
LightGBM (Light Gradient Boosting Machine) là một framework gradient boosting được phát triển bởi Microsoft, tối ưu hóa cho tốc độ và hiệu suất cao. LightGBM sử dụng các kỹ thuật đột phá để cải thiện tốc độ huấn luyện và giảm sử dụng bộ nhớ.

### **Ưu điểm:**
- Tốc độ huấn luyện cực nhanh (nhanh hơn XGBoost 10-20 lần)
- Hiệu suất cao, thường tương đương hoặc tốt hơn XGBoost
- Xử lý tốt dữ liệu lớn và nhiều chiều

### **Nhược điểm:**
- Dễ bị overfitting với dữ liệu nhỏ.
- Nhạy cảm với siêu tham số.


## Bước 1 - Import các thư viện cần thiết

### 1.1. Cài thư viện lightgbm

In [16]:
!pip install lightgbm

### 1.2. Import các thư viện cơ bản khác

In [17]:
# Thư viện cơ bản
import numpy as np
import pandas as pd
import os

# Thư viện LightGBM
import lightgbm as lgb
from lightgbm import LGBMRegressor

# Thư viện sklearn cho đánh giá
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Thư viện để lưu mô hình
import joblib

## Bước 2 - Đọc dữ liệu đã tiền xử lý

Đọc dữ liệu từ các file CSV đã được tiền xử lý.

In [18]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập train (đã được chuẩn hóa):")
train_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (3124, 13)

5 dòng đầu tiên của tập train (đã được chuẩn hóa):


,country_name,country_code,year,population,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,labor_force
0,Denmark,DNK,2017,-0.203264,-0.418953,81.102439,1.682694,-0.047741,1.673510,0.642797,0.754689,0.118599,0.069223
1,"Korea, Dem. People's Rep.",PRK,2017,-0.056076,-0.530921,73.034000,-0.412271,0.028651,-0.245951,-1.294730,0.439985,-0.216848,2.087276
2,Madagascar,MDG,2008,-0.091998,1.007790,61.992000,-0.607170,0.547296,-1.651029,-2.194047,-2.540432,-0.528445,2.542268
3,Greece,GRC,2018,-0.166799,-0.945727,81.787805,0.170491,-0.209157,1.356075,0.642797,0.754689,0.161554,-1.033944
4,South Sudan,SSD,2019,-0.169071,1.000983,58.129000,-0.412271,0.028651,-1.375450,-2.593358,-2.691194,-0.278341,1.279234


## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`

In [19]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

THÔNG TIN CÁC TẬP DỮ LIỆU
Số lượng đặc trưng: 10

Các đặc trưng được sử dụng:
  1. year
  2. population
  3. pop_growth
  4. gdp_per_capita
  5. gdp_growth
  6. sanitation
  7. electricity
  8. water_access
  9. co2_emissions
  10. labor_force

Kích thước X_train: (3124, 10)
Kích thước y_train: (3124,)


## Bước 4 - Xây dựng và huấn luyện mô hình LightGBM

### 4.1. Sử dụng RandomizedSearchCV với 5-Fold Cross-Validation để tìm kiếm siêu tham số tối ưu cho mô hình LightGBM.

In [20]:
# Định nghĩa không gian siêu tham số để tìm kiếm
from scipy.stats import randint, uniform

param_distributions_lgbm = {
    "n_estimators": randint(200, 1200),
    "learning_rate": uniform(0.005, 0.05),  
    "num_leaves": randint(20, 300),
    "max_depth": randint(3, 20),
    "min_child_samples": randint(5, 50),
    "min_child_weight": uniform(1e-3, 0.1),
    "reg_alpha": uniform(0.0, 1.0),   
    "reg_lambda": uniform(0.0, 1.0),  
    "subsample": uniform(0.5, 0.5),         
    "colsample_bytree": uniform(0.5, 0.5),  
    "min_split_gain": uniform(0.0, 0.05),
}

# Tạo RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=LGBMRegressor(random_state=42, verbose=-1),
    param_distributions=param_distributions_lgbm,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    n_iter=60
)

# Thực hiện tìm kiếm
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


KeyboardInterrupt: 

### 4.2. Hiển thị siêu tham số tối ưu và đánh giá hiệu suất của mô hình tốt nhất.

In [ ]:
# Lấy mô hình tốt nhất và tham số tối ưu
best_lgbm = random_search.best_estimator_
best_params = random_search.best_params_

print("="*60)
print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params.items():
    print(f"  {param:20s}: {value}")

# Thông tin về mô hình tối ưu
print(f"\nSố lượng iterations: {best_lgbm.n_estimators}")
print(f"Number of leaves: {best_lgbm.num_leaves}")
print(f"Learning rate: {best_lgbm.learning_rate}")
print(f"Max depth: {best_lgbm.max_depth}")

SIÊU THAM SỐ TỐI ƯU
  colsample_bytree    : 0.7414853085811328
  learning_rate       : 0.0385263932833753
  max_depth           : 16
  min_child_samples   : 7
  min_child_weight    : 0.07961038245827129
  min_split_gain      : 0.0016836805159890058
  n_estimators        : 1061
  num_leaves          : 69
  reg_alpha           : 0.42972727587562565
  reg_lambda          : 0.9572984840006148
  subsample           : 0.7646352317409129

Số lượng iterations: 1061
Number of leaves: 69
Learning rate: 0.0385263932833753
Max depth: 16


### 4.3. Đánh giá mô hình tối ưu trên tập train

In [ ]:
# Dự đoán trên tập train và validation
y_train_pred = best_lgbm.predict(X_train)

# Tính toán các metrics cho tập train
train_mae = mean_absolute_error(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: LightGBM:")
print(f"RMSE loss: {train_rmse:.4f}")
print(f"MAE loss: {train_mae:.4f}")
print(f"R2 score: {train_r2:.4f}")

KẾT QUẢ MÔ HÌNH TỐI ƯU
Model: LightGBM:
RMSE loss: 0.1561
MAE loss: 0.1088
R2 score: 0.9997


### 4.4. Lưu mô hình đã huấn luyện

Lưu mô hình LightGBM đã tối ưu vào file để sử dụng sau này.

In [ ]:
# Tạo thư mục model nếu chưa có
os.makedirs('../models/6_lightgbm', exist_ok=True)

# Lưu mô hình
model_path = '../models/6_lightgbm/lightgbm.pkl'
joblib.dump(best_lgbm, model_path)

['../models/6_lightgbm/lightgbm.pkl']

## Kết luận

### Tổng kết:
- Mô hình LightGBM đã được xây dựng và tối ưu hóa thành công
- RandomizedSearchCV với 5-Fold CV đã tìm được bộ siêu tham số tối ưu
- Mô hình được đánh giá dựa trên 3 metrics: MAE, RMSE và R2